# Delivery optimization

In this notebook it will be solved a knapsack problem with the library [PulP](https://coin-or.github.io/pulp).

There is one warehouse with hardware and far away one new office without hardware. This office needs to be equipped. For this exists **transporters** like trucks. 

The job is to find the most efficient loading for the transporters with hardware for the **first** delivery.  
All available hardware items have a **weight**, a **utility** value for the new office and a maximum of required **count** for the new office. 
The transporters have a **maximum load capacity**, which limits the weight of their **driver and items**.

This exercise comes from a [coding challenge](https://www.get-in-it.de/coding-challenge) from [get in IT](https://www.get-in-it.de).  
There are the following specific example values for this: 

- truck count: 2
- truck 1 max capacity: 1100kg
- truck 2 max capacity: 1100kg  
- driver 1 weight: 72.4kg
- driver 2 weight: 85.7kg
- shown table of items below


The following table of items is used in the code by a csv file named `items.csv`  
*Notice: In the code the names of the items can be different, because the originals from the csv file are in german.*  


| name                  | max count | weight in gram | utility per unit |
|-----------------------|-----------|----------------|------------------|
|Notebook office 13"    |       205 |           2451 |               40 |
|Notebook office 14"    |       420 |           2978 |               35 |
|Notebook outdoor       |       450 |           3625 |               80 |
|Mobile phone office    |        60 |            717 |               30 |
|Mobile phone outdoor   |       157 |            988 |               60 |
|Mobile phone heavy duty|       220 |           1220 |               65 |
|Tablet office small    |       620 |           1405 |               40 |
|Tablet office big      |       250 |           1455 |               40 |
|Tablet outdoor small   |       540 |           1690 |               45 |
|Tablet outdoor big     |       370 |           1980 |               68 |

So lets take a look how to get the greatest sum of utility in the trucks. 

### How to solve this problem?

The knapsack problem is based on the objects which should be loaded.  
So there are always whole parts and not just a half or a quarter of it.  

The knapsack problem can be modeled as an integer programming optimization.  
Integer programming is a mathematical optimization program in which all or some variables are integers instead of continues point numbers.  

For modelling a integer programming optimization there must be defined the following:
- Decision variables which will be optimized
- A objective function which are maximized or minimized
- Some constraints to restrict the decision variables 


**In this use case**

- The decision variables are the items to load.  
- Our objective function is the total sum of utility for all items on all trucks.  
- Constraints can be defined from the max load capacity and the max count of item type. 

### How it works?

#### Quick overview

The standard method to solve an integer programming is called "Branch and Bound". 

Because an integer programming model is harder to optimize than a linear programming model, there will be used a linear programming optimization as help.  

**Bound**  
The linear model can optimize the objective function with continues numbers, so its max value will always be greater than in a integer programming model.  
This can be seen as the upper bound.  
A lower bound exists also to set the required minimum for solutions. By example at start to 0 for all variables. 

**Branch**  
While computing combinations to reach the upper bound, there will be some found that fulfill the constraints.  
With this solutions the branching will repeatally correct the upper and lower bound to let their distance get smaller and smaller.  
The upper bound will decrease, because it is to high for the integer programming.  
The lower bound will increase with each best minimum solution found, so that all solutions with lower value can be removed.  
&nbsp;

There are different solvers available for problems like that, which could process this for us.  
In this notebook we use PulP and let it choose the right one. 

### Why PulP?

PulP is one of many libraies which offer a linear programming API for linear problems.  
The reason for using PulP is, that the decision variables can be set as integers instead continues point numbers.  
This is essential for the integer programming problem. 

### Required imports

In [1]:
# python=3.7.6

# pandas=1.0.1
import pandas as pd
# pulp=2.4
from pulp import *

## All changeable parameters

**Extending the example from the exercise, it can be chosen fewer or more transporters, other maximum capacities (for each truck), other driver weights and other values for the items.**  

Make sure that the columns in the csv contains the right values.  
If the column names are not the required ones, a mapping is offered. See below at column_mapping_required and columns_mapping.  

Required column names (order not necessary): **name** ; **max_count** ; **weight** ; **utility**  

Notice to set the right measuring unit 'kg' or 'gram' for the max loadings (max capacities), driver weight and item weight. 

In [2]:
# trucks count and max loading weight
trucks_max_loadings = [1100, 1100]
# can be 'kg' or 'gram'
trucks_measuring_unit = 'kg'
# drivers weight
drivers_weights = [72.4, 85.7]
# can be 'kg' or 'gram'
drivers_measuring_unit = 'kg'

# path to the csv file with all items
items_file = './items.csv'
# if False, the column names in the csv must be name, max_count, weight, utility
# if True, the column names from csv will be changed by columns_mapping below
column_mapping_required = True
# the csv column names mapped to name, max_count, weight and utility
# "<required column>" : ""<mapping csv column>""
columns_mapping = {
    'name':      'name', 
    'max_count': 'required count', 
    'weight':    'weight in gram', 
    'utility':   'utility'
}

# delimiter of the csv columns
csv_delimiter = ';'


if len(trucks_max_loadings) != len(drivers_weights):
    print("The count of trucks must be equal to the count of drivers!")


## Calculate the possible loads for the trucks

If necessary, converting kg to gram for the trucks max loading and drivers weight.

In [3]:
if trucks_measuring_unit == 'kg':
    trucks_max_loadings_gram = [max_load*1000 for max_load in trucks_max_loadings]
else:
    trucks_max_loadings_gram = trucks_max_loadings

if drivers_measuring_unit == 'kg':
    drivers_weights_gram = [weight*1000 for weight in drivers_weights]
else:
    drivers_weights_gram = drivers_weights

trucks_max_loadings_gram

[1100000, 1100000]

Subtract the drivers weights from trucks max loading weight.

In [4]:
trucks_possible_loads = [max_load - driver_weight 
                         for max_load,driver_weight 
                         in zip(trucks_max_loadings_gram, drivers_weights_gram)]

trucks_possible_loads

[1027600.0, 1014300.0]

## Exploring the items

Read the data from csv file into a dataframe by pandas.

In [5]:
df_items = pd.read_csv(items_file, delimiter=csv_delimiter, decimal='.')
df_items.head(10)

,name,required count,weight in gram,utility
0,"Notebook Büro 13""",205,2451,40
1,"Notebook Büro 14""",420,2978,35
2,Notebook outdoor,450,3625,80
3,Mobiltelefon Büro,60,717,30
4,Mobiltelefon Outdoor,157,988,60
5,Mobiltelefon Heavy Duty,220,1220,65
6,Tablet Büro klein,620,1405,40
7,Tablet Büro groß,250,1455,40
8,Tablet outdoor klein,540,1690,45
9,Tablet outdoor groß,370,1980,68


### Map the column names if required

In [6]:
if column_mapping_required and columns_mapping is not None:
    df_items.rename(columns = {origin_name : new_name for new_name,origin_name in columns_mapping.items()}, inplace=True)
df_items.head(10)

,name,max_count,weight,utility
0,"Notebook Büro 13""",205,2451,40
1,"Notebook Büro 14""",420,2978,35
2,Notebook outdoor,450,3625,80
3,Mobiltelefon Büro,60,717,30
4,Mobiltelefon Outdoor,157,988,60
5,Mobiltelefon Heavy Duty,220,1220,65
6,Tablet Büro klein,620,1405,40
7,Tablet Büro groß,250,1455,40
8,Tablet outdoor klein,540,1690,45
9,Tablet outdoor groß,370,1980,68


### Calculate a value of efficiency by utility / weight

*Notice: This step is still just exploring of the data. It is not necessary for the solution.*  

To find out which items are the best choices and should be nearly all on the delivery, the efficiency is a good approach. 

In [7]:
items_efficiency = {item['name'] : item['utility'] / item['weight'] for _, item in df_items.iterrows()}

print("Efficients ordered\n")
items_efficiency_ordered = dict(sorted(items_efficiency.items(), key=lambda e: e[1], reverse=True))

# for more readability its multiplied by 1000
for name, efficiency in items_efficiency_ordered.items():
    print("{:<25}: {}".format(name, round(efficiency*1000, 2)))

Efficients ordered

Mobiltelefon Outdoor     : 60.73
Mobiltelefon Heavy Duty  : 53.28
Mobiltelefon Büro        : 41.84
Tablet outdoor groß      : 34.34
Tablet Büro klein        : 28.47
Tablet Büro groß         : 27.49
Tablet outdoor klein     : 26.63
Notebook outdoor         : 22.07
Notebook Büro 13"        : 16.32
Notebook Büro 14"        : 11.75


# Start with pulp action

## Create a new LpProblem from pulp.  
The LpProblem should maximize a objective function, which will be set later.  
There is also the possibility to instead minimize an objective function maybe for an error problem. 

In [8]:
prob = LpProblem("Delivery_Problem",LpMaximize)

## Create LpVariables from pulp for optimizing.  
The values of this variables will be changed by the solver included in pulp to optimize the objective function.  

In our case a LpVariable has the name of an item, the optional lowBound is set to 0, because no less than 0 items can be taken, and the category set to Integer to get no half or quarter pieces.  

Like following it can be create each variable separately with different parameters or a dict of multiple variables with the same parameters.  

Example for a single variable: 
> `variable_1 = LpVariable('<variable_name>', lowBound=0, upBound=None, cat='Integer')`

Example for a dict of multiple variables (a dict have a main/upper name to subdivide): 
> `variables_dict = LpVariable.dicts("<main_name>",<list_of_variables_names>,lowBound=0,cat='Integer')`  

**We create for each truck a own dict consisting of all items.** 

In [9]:
items_names = df_items['name'].tolist()

# Set variables to optimize as items count, lower bound = 0 and as Integer numbers
# for single variables make: variable_1 = LpVariable('<variable_name>', lowBound=0, upBound=None, cat='Integer')
# for multiple variables make: LpVariable.dicts("<main_name>",<list_of_variables_names>,lowBound=0,cat='Integer')

# create optimizeable variables for each item for each truck as a list of dicts
lp_trucks_items = [LpVariable.dicts("Truck {}".format(truck_number), items_names, lowBound=0, cat='Integer') 
                   for truck_number in range(len(trucks_possible_loads))]


## Adding main objective function

Construct a objective function and add it to the LpProblem.  
The objective function will be calculated repeatedly to adjust the LpVariables in the right way to maximize itself.  

For the objective function we calculate the total sum of utility for the items on the trucks.  
To do this, we multiply the utility value of the item by its count from the LpVariable.  
We do that with each item on each truck and get the sum of all. 

In [10]:
# Adding main objective function

# multiply the utility of the item with the count in the optimizeable variables
# for each item in each truck and get the sum as objective value
prob += lpSum([
    item['utility'] * lp_truck_items[item['name']]
    for _,item in df_items.iterrows() 
    for lp_truck_items in lp_trucks_items
])


## Adding constraints

After adding the objective function to the LpProblem, it can be added some constraints in the same way. 

### Adding max capacity constraints

We have a limit of load for each truck, what we can implement as constraints.  
Instead of taking the total sum as in the objective function, here we create a separate constraint for each truck.  
To do this, we multiply the weight of the item by its count from the LpVariable.  
We do that with each item on **one** truck and get the sum of its weight.  
Then we add a new constraint where the sum of weight must be lower or equal to the truck max possible load (which we got further up).  
Repeat for all trucks. 

In [11]:
# Adding weight constraints

# for each truck get the possible load
for truck_number, truck_possible_load in enumerate(trucks_possible_loads):
    lp_truck_items = lp_trucks_items[truck_number]    
    # multiply the weight of the item with its count on the truck
    lp_sum = lpSum([float(item['weight']) * lp_truck_items[item['name']] for _,item in df_items.iterrows()])
    # set the constraint, that the sum of weights on the truck must be lower or equal to his possible load
    prob += lp_sum <= truck_possible_load

### Adding item max count constraints

Like the max possible load above it exists also a limit by the max count of each item type.  
This limit applies one item type over all trucks.  
So we go through the item list and get the count of it from each truck by the LpVariables in `lp_trucks_items`.  
We sum the count over all trucks and add a new constraint where it must be lower or equal to the defined max count.  
Repeat for all items. 

In [12]:
# Adding count constraints
# for each item type get the sum of count on each truck
# set the constraint, that the sum of count is lower or equal to the max count
for _, item in df_items.iterrows():
    total_item_count = sum([lp_truck_items[item['name']] for lp_truck_items in lp_trucks_items])
    prob += total_item_count <= item['max_count']

## Solving
### Try to solve the definded problem

It can be set a certain solver from the pulp library.  
However, we run the default solver which is automatically selected by the structure of the defined problem.  

**Here the algorithmus do optimizing the LpVariables.**

In [13]:
# Use default solver which will be selected by the problem structure
prob.solve()

1

Show the status of solving. 

In [14]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

Status: Optimal


## Present the results and consider the validity
### Convert results to own data structure

Because of the subdivide by upper name in the dicts has each LpVariable name a string as prefix like `"Truck_1_"`.  
Furthermore the library has replaced all whitespaces in the variable names by underscores.  
So we can not use the original item name as key to get the value of a item.  

To make easy access we make a own result list for each truck.  
In this list we add a dictionary for all items on the respective truck.  
Because the division for each truck, we can remove the prefix on all variables.  
Just the replaced whitespaces are taken and handled later.  
The result values keep. 

In [15]:
# get optimized variables
result_variables = prob.variables()
# create a result list for the trucks
result_trucks_items = []
# for each truck add his items
for truck_number in range(len(trucks_possible_loads)):
    prefix_len = len("Truck_{}_".format(truck_number))
    # create a result item dict for this truck
    truck_items = {var.name[prefix_len:] : var.varValue                      # remove the prefix of truck in the name
                   for var in result_variables 
                   if var.name.startswith("Truck_{}_".format(truck_number))} # get only the items from the current truck
    # add count of items for this truck to result list
    result_trucks_items.insert(truck_number, truck_items)
    
result_trucks_items

[{'Mobiltelefon_Büro': 52.0,
  'Mobiltelefon_Heavy_Duty': 220.0,
  'Mobiltelefon_Outdoor': 0.0,
  'Notebook_Büro_13"': 0.0,
  'Notebook_Büro_14"': 0.0,
  'Notebook_outdoor': 0.0,
  'Tablet_Büro_groß': 0.0,
  'Tablet_Büro_klein': 509.0,
  'Tablet_outdoor_groß': 0.0,
  'Tablet_outdoor_klein': 4.0},
 {'Mobiltelefon_Büro': 8.0,
  'Mobiltelefon_Heavy_Duty': 0.0,
  'Mobiltelefon_Outdoor': 157.0,
  'Notebook_Büro_13"': 0.0,
  'Notebook_Büro_14"': 0.0,
  'Notebook_outdoor': 0.0,
  'Tablet_Büro_groß': 0.0,
  'Tablet_Büro_klein': 86.0,
  'Tablet_outdoor_groß': 370.0,
  'Tablet_outdoor_klein': 0.0}]

### Show each truck with driver and item count

Print the result for each truck.  
How much weight has the driver.  
What types and how many items should be loaded.  

*Notice: The Assignment for trucks and drivers is set by the indices of lists.*

In [16]:
print("Result count for each item")

for truck_number, result_truck_items in enumerate(result_trucks_items):
    # increase the number of truck by 1 to skip the index 0 (only for presentation)
    # also divide the weight of driver by 1000 to get kg from gram
    print("\nTruck {} with driver weight {}kg:".format(truck_number+1,drivers_weights_gram[truck_number]/1000))
    for item_name, item_count in result_truck_items.items():
        # show only the items which counts are greater 0
        if item_count > 0:
            # replace underscores from pulp with whitespaces for better readability
            print("{:<25} = {:>10}".format(item_name.replace('_',' '),item_count))

Result count for each item

Truck 1 with driver weight 72.4kg:
Mobiltelefon Büro         =       52.0
Mobiltelefon Heavy Duty   =      220.0
Tablet Büro klein         =      509.0
Tablet outdoor klein      =        4.0

Truck 2 with driver weight 85.7kg:
Mobiltelefon Büro         =        8.0
Mobiltelefon Outdoor      =      157.0
Tablet Büro klein         =       86.0
Tablet outdoor groß       =      370.0


### Show possible load of trucks

In [17]:
for truck_number, truck_possible_load in enumerate(trucks_possible_loads):
    print("Allowed weight for truck {} (with included driver): {} gram or {} kg".format(
        truck_number+1,
        truck_possible_load, 
        truck_possible_load / 1000))

Allowed weight for truck 1 (with included driver): 1027600.0 gram or 1027.6 kg
Allowed weight for truck 2 (with included driver): 1014300.0 gram or 1014.3 kg


### Show total weights of trucks containing items

*Notice:  
The library sorted the variables by names and changed their indices.  
So they have different indices as the origin.  
To access one use the name as key.*  

To get the total sum of weight for each truck and the chosen count of items, we go through each truck from our own result list `result_trucks_items`.  
We multiply the weight of item with the count from truck results.  

Because the library has replaced all whitespaces in the variable names by underscores, we access the result value by do this with the origin name too and use it as key.  
*Notice: We can not just replace all underscores with whitespaces, because a origin name could contain a underscore too.*

In [18]:
# for each truck
for truck_number, result_truck_items in enumerate(result_trucks_items):
    # calculate the total sum of weight for each item
    total_truck_weight = sum([
        item_count * item['weight'] 
        for item_name,item_count in result_truck_items.items() 
        for _,item in df_items.iterrows() 
        # the item names from pulp have replaced whitespaces ' ' with underscore '_'
        # to compare we must do this too
        if item['name'].replace(' ','_') == item_name 
    ])
    print("Total weight of truck {}: {} gram or {} kg".format(truck_number+1, total_truck_weight, total_truck_weight/1000))


Total weight of truck 1: 1027589.0 gram or 1027.589 kg
Total weight of truck 2: 1014282.0 gram or 1014.282 kg


### Show the total value of utility over all trucks and items

In [19]:
# print the end value of the objective function, in our case the sum of utility from all picked items
result_utility = value(prob.objective)
print("The total utility value (rounded to 2 decimal places) of picked items in both trucks is: {}".
      format(round(result_utility,2)))

The total utility value (rounded to 2 decimal places) of picked items in both trucks is: 74660.0
